In [1]:
import warnings
import tensorflow as tf
import numpy as np
import pandas as pd
import json
#from modules.CARLA import carla

seed = 44

tf.random.set_seed(seed)
np.random.seed(seed)

In [2]:
# from carla.data.catalog import CsvCatalog
# import json

# with open('../data/adult_constraints.json', 'r') as f:
#     constraints = json.load(f)

# continuous = constraints['continuous']
# categorical = constraints['categorical']
# immutable = constraints['immutable']
# columns_order = constraints['features_order']

# dataset = CsvCatalog(file_path="../data/adult_prep.csv",
#                     continuous=continuous,
#                     categorical=categorical,
#                     immutables=immutable,
#                     target='income')
# dataset.df.to_csv("../data/adult_prep_test.csv",index=False)



# X_train = dataset.df_train
# Y_train = dataset.df_train
# X_test = dataset.df_test
# Y_test = dataset.df_test
# X_train

In [3]:
import json
import sklearn
from sklearn.model_selection import train_test_split

with open('../data/adult_constraints.json', 'r') as f:
    constr = json.load(f)

dataset = pd.read_csv("../data/adult_cleaned.csv")

X_train, X_test, Y_train, Y_test = train_test_split(dataset[constr['features_order_after_split']], dataset['income'], test_size=0.2, random_state=seed)
X_train.columns

Index(['capital.gain', 'capital.loss', 'hours.per.week', 'age',
       'education.num', 'workclass_?', 'workclass_Federal-gov',
       'workclass_Local-gov', 'workclass_Never-worked', 'workclass_Private',
       'workclass_Self-emp-inc', 'workclass_Self-emp-not-inc',
       'workclass_State-gov', 'workclass_Without-pay',
       'marital.status_Divorced', 'marital.status_Married-AF-spouse',
       'marital.status_Married-civ-spouse',
       'marital.status_Married-spouse-absent', 'marital.status_Never-married',
       'marital.status_Separated', 'marital.status_Widowed', 'occupation_?',
       'occupation_Adm-clerical', 'occupation_Armed-Forces',
       'occupation_Craft-repair', 'occupation_Exec-managerial',
       'occupation_Farming-fishing', 'occupation_Handlers-cleaners',
       'occupation_Machine-op-inspct', 'occupation_Other-service',
       'occupation_Priv-house-serv', 'occupation_Prof-specialty',
       'occupation_Protective-serv', 'occupation_Sales',
       'occupation_Tech

In [4]:
Y_train

20789     >50K
7665     <=50K
7003     <=50K
3368     <=50K
15072    <=50K
         ...  
16955    <=50K
25773     >50K
27377    <=50K
3491     <=50K
14100    <=50K
Name: income, Length: 26048, dtype: object

In [5]:

X_train = X_train.to_numpy()
Y_train = Y_train
Y_train = pd.get_dummies(Y_train).to_numpy()
Y_train

array([[0, 1],
       [1, 0],
       [1, 0],
       ...,
       [1, 0],
       [1, 0],
       [1, 0]], dtype=uint8)

In [6]:
X_test = X_test.to_numpy()
Y_test = Y_test
Y_test = pd.get_dummies(Y_test).to_numpy()
Y_test

array([[1, 0],
       [1, 0],
       [1, 0],
       ...,
       [1, 0],
       [1, 0],
       [0, 1]], dtype=uint8)

In [7]:
X_train.shape

(26048, 85)

In [8]:
Y_train.shape

(26048, 2)

In [9]:
# sklearn minmaxscaler
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [10]:
constr['features_count_split_without_target']

85

In [11]:
class_weights = np.round(1 - Y_train.sum(axis=0) / Y_train.sum(), 4)
class_weights

array([0.2397, 0.7603])

In [12]:
tf_class_weights = {0: class_weights[0], 1: class_weights[1]}
tf_class_weights

{0: 0.2397, 1: 0.7603}

In [61]:
from tensorflow import keras


model = tf.keras.Sequential()
model.add(tf.keras.layers.Input((constr['features_count_split_without_target'],)))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dropout(0.4))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(2))
model.add(tf.keras.layers.Softmax())


model.compile(
    optimizer='adam',
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [62]:
model.fit(
    X_train, 
    Y_train,
    epochs=50,
    batch_size=512,
    validation_data=(X_test, Y_test),
    shuffle=True,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(patience=10)
    ],
    verbose=1,
    class_weight=tf_class_weights
)

Epoch 1/50
51/51 [==============================] - 3s 27ms/step - loss: 0.2299 - accuracy: 0.6171 - val_loss: 0.5132 - val_accuracy: 0.7321
Epoch 2/50
51/51 [==============================] - 1s 22ms/step - loss: 0.1867 - accuracy: 0.7388 - val_loss: 0.4608 - val_accuracy: 0.7408
Epoch 3/50
51/51 [==============================] - 1s 20ms/step - loss: 0.1753 - accuracy: 0.7494 - val_loss: 0.4495 - val_accuracy: 0.7503
Epoch 4/50
51/51 [==============================] - 1s 20ms/step - loss: 0.1681 - accuracy: 0.7579 - val_loss: 0.4214 - val_accuracy: 0.7723
Epoch 5/50
51/51 [==============================] - 1s 20ms/step - loss: 0.1666 - accuracy: 0.7573 - val_loss: 0.4325 - val_accuracy: 0.7619
Epoch 6/50
51/51 [==============================] - 1s 20ms/step - loss: 0.1628 - accuracy: 0.7678 - val_loss: 0.4220 - val_accuracy: 0.7801
Epoch 7/50
51/51 [==============================] - 1s 20ms/step - loss: 0.1589 - accuracy: 0.7769 - val_loss: 0.4027 - val_accuracy: 0.7915
Epoch 8/50
51

In [13]:
model.save('../models/adult_NN.h5', overwrite=True, save_format='h5')

In [14]:
model.save('../models/adult_NN', overwrite=True)
model_test = tf.keras.models.load_model('../models/adult_NN.h5')
model_test.predict(X_test[0:2], verbose=1)

INFO:tensorflow:Assets written to: ../models/adult_NN\assets


INFO:tensorflow:Assets written to: ../models/adult_NN\assets


1/1 [==============================] - 0s 88ms/step


array([[8.4801334e-01, 1.5198670e-01],
       [9.9952066e-01, 4.7937059e-04]], dtype=float32)

In [16]:
model.load_weights('../models/adult_NN.h5')

In [17]:
X_test[0].shape

(85,)

In [18]:
model.predict(X_test[0:2], verbose=1)


1/1 [==============================] - 0s 61ms/step


array([[9.321678e-01, 6.783224e-02],
       [9.990681e-01, 9.319379e-04]], dtype=float32)

RandomForest

In [13]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from joblib import dump, load

In [41]:
best_score = 0


for crit in ['gini', 'entropy']:
    for max_depth in [10, 30, None]: 
        for estimators in [150, 200]:
            for max_features in ['sqrt', 'log2']:
                for min_samples_leaf in [1,2,3]:
                    for min_samples_split in [2,3,4]:
                        for min_impurity_decrease in [0, 0.001, 0.01, 0.1]:
                            classifier = RandomForestClassifier(
                                random_state=seed, 
                                criterion=crit, 
                                n_estimators=estimators,
                                verbose=0,
                                class_weight='balanced',
                                min_impurity_decrease=min_impurity_decrease,
                                min_samples_leaf=min_samples_leaf,
                                min_samples_split=min_samples_split,
                                max_depth=max_depth,
                                n_jobs=3,
                                max_features=max_features
                                )
                            classifier.fit(X_train, Y_train)
                            score = classifier.score(X_test, Y_test)
                            if score > best_score:
                                best_score = score
                                print(f'New Best! Crit: {crit}, Max_depth: {max_depth}, Max_feat: {max_features}, Min_sampl_leaf: {min_samples_split} \
MIn_samples_split {min_samples_split}, Min_impurity_dec: {min_impurity_decrease}, Estimators: {estimators} Score: {score}')

New Best! Crit: gini, Max_depth: 10, Max_feat: sqrt, Min_sampl_leaf: 2 MIn_samples_split 2, Min_impurity_dec: 0, Estimators: 150 Score: 0.6961461692000614
New Best! Crit: gini, Max_depth: 10, Max_feat: sqrt, Min_sampl_leaf: 3 MIn_samples_split 3, Min_impurity_dec: 0, Estimators: 150 Score: 0.699677567941041
New Best! Crit: gini, Max_depth: 30, Max_feat: sqrt, Min_sampl_leaf: 2 MIn_samples_split 2, Min_impurity_dec: 0, Estimators: 150 Score: 0.8252725318593582
New Best! Crit: gini, Max_depth: None, Max_feat: sqrt, Min_sampl_leaf: 2 MIn_samples_split 2, Min_impurity_dec: 0, Estimators: 150 Score: 0.839091048671887
New Best! Crit: gini, Max_depth: None, Max_feat: sqrt, Min_sampl_leaf: 3 MIn_samples_split 3, Min_impurity_dec: 0, Estimators: 150 Score: 0.8453861507753724
New Best! Crit: gini, Max_depth: None, Max_feat: sqrt, Min_sampl_leaf: 4 MIn_samples_split 4, Min_impurity_dec: 0, Estimators: 150 Score: 0.8466144633809305


In [54]:
classifier = RandomForestClassifier(
                random_state=seed, 
                criterion='gini', 
                n_estimators=150,
                verbose=0,
                class_weight='balanced',
                min_impurity_decrease=0,
                max_depth=None,
                n_jobs=3,
                max_features='sqrt'
            )
classifier.fit(X_train, Y_train)
classifier.score(X_test, Y_test)

0.839091048671887

In [55]:
dump(classifier, '../models/adult_RF.joblib')

['../models/adult_RF.joblib']

In [56]:
clf = load('../models/adult_RF.joblib')

In [57]:
print(Y_test[0:5])
clf.predict(X_test[0:5])

[[1 0]
 [1 0]
 [1 0]
 [0 1]
 [0 1]]


array([[1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [0, 1]], dtype=uint8)

In [58]:
import pickle

with open('../models/adult_RF.pkl', 'wb') as f:
    pickle.dump(classifier, f)

with open('../models/adult_RF.pkl', 'rb') as f:
    clf_pickl = pickle.load(f)

pred = clf_pickl.predict(X_test[0:5])
type(pred)

numpy.ndarray

In [59]:
clf_pickl.predict_proba(X_test[0:10])[0]

array([[0.03333333, 0.96666667],
       [0.00666667, 0.99333333],
       [0.        , 1.        ],
       [0.01848003, 0.98151997],
       [0.95333333, 0.04666667],
       [0.34464258, 0.65535742],
       [0.00666667, 0.99333333],
       [0.37333333, 0.62666667],
       [0.79333333, 0.20666667],
       [0.18460443, 0.81539557]])